# Benchmarking with OLS
First we create a class for using ordinary least squares to fit the model and to predict outputs for unknown inputs. Numpy has some great linear algebra functions that we can use to help us quickly find the beta value for minimizing the sum of squared errors. We create two helper functions: *gradientDescent* and *computeGradient*, to help us implement gradient descent with our model. 

In [ ]:
# import our handy dandy libraries 
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
import pandas

# increase the width of boxes in the notebook file (this is only cosmetic)
np.set_printoptions(linewidth=180)

In [ ]:
class OrdinaryLeastSquaresGradient:
        
    # fit the model to the data
    def fit(self, X, y, h, tolerance, maxIterations):
        self.n = X.shape[0]
        self.d = X.shape[1]
        
        # save the training data
        self.data = np.hstack((np.ones([self.n, 1]), X))
        
        # save the training labels
        self.outputs = y
        
        # find the beta values that minimize the sum of squared errors via gradient descent
        X = self.data
        L = lambda beta: ((X @ beta).T - y.T) @ (X @ beta - y)
        self.beta = self.gradientDescent(L, (self.d + 1) * [0], h, tolerance, maxIterations)
                
    # predict the output from testing data
    def predict(self, X):
        # initialize an empty matrix to store the predicted outputs
        yPredicted = np.empty([X.shape[0],1])
        
        # append a column of ones at the beginning of X
        X = np.hstack((np.ones([X.shape[0],1]), X))
        
        # apply the function f with the values of beta from the fit function to each testing datapoint (rows of X)
        for row in range(X.shape[0]):
            yPredicted[row] = self.beta @ X[row,]
            
        return yPredicted

    # run gradient descent to minimize the loss function
    def gradientDescent(self, f, x0, h, tolerance, maxIterations):
        # set x equal to the initial guess
        x = x0

        # take up to maxIterationsa number of steps
        for counter in range(maxIterations):
            # update the gradient
            gradient = self.computeGradient(f, x, h)

            # stop if the norm of the gradient is near 0
            if np.linalg.norm(gradient) < tolerance:
                print('Gradient descent took', counter, 'iterations to converge')
                print('The norm of the gradient is', np.linalg.norm(gradient))
                # return the approximate critical value x
                return x

            # if we do not converge, print a message
            elif counter == maxIterations-1:
                print("Gradient descent failed")
                print('The gradient is', gradient)
                # return x, sometimes it is still pretty good
                return x

            # take a step in the opposite direction as the gradient
            x -= h*gradient

            
    # compute the gradient
    def computeGradient(self, f, x, h):
        n = len(x)
        gradient = np.zeros(n)

        for counter in range(n):
            xUp = x.copy()
            xUp[counter] += h
            gradient[counter] = (f(xUp) - f(x))/h

        return gradient

# With Momentum
We can improve our gradient descent by adding momentum. Momentum will imporve our algorithm by taking more direct (more informed) steps toward the minimum and so, we can converge with fewer iterations. This method introduces a hyperparameter B. After some hyperparameter testing (and googling) a value of B=0.9 was found to be an ideal value for momentum. 

In [ ]:
class OrdinaryLeastSquaresGradientMomentum:
        
    # fit the model to the data
    def fit(self, X, y, h, tolerance, maxIterations):
        self.n = X.shape[0]
        self.d = X.shape[1]
        
        # save the training data
        self.data = np.hstack((np.ones([self.n, 1]), X))
        
        # save the training labels
        self.outputs = y
        
        # find the beta values that minimize the sum of squared errors via gradient descent
        X = self.data
        L = lambda beta: ((X @ beta).T - y.T) @ (X @ beta - y)
        self.beta = self.gradientDescent(L, (self.d + 1) * [0], h, tolerance, maxIterations)
                
    # predict the output from testing data
    def predict(self, X):
        # initialize an empty matrix to store the predicted outputs
        yPredicted = np.empty([X.shape[0],1])
        
        # append a column of ones at the beginning of X
        X = np.hstack((np.ones([X.shape[0],1]), X))
        
        # apply the function f with the values of beta from the fit function to each testing datapoint (rows of X)
        for row in range(X.shape[0]):
            yPredicted[row] = self.beta @ X[row,]
            
        return yPredicted

    # run gradient descent to minimize the loss function
    def gradientDescent(self, f, x0, h, tolerance, maxIterations):
        # set x equal to the initial guess
        x = x0
        # set the value for momentum
        B = 0.9

        # vdL will be used with B for updating the values in x
        # we initialize with zeros the same length of x
        vdL = np.zeros(len(x))
       

        # take up to maxIterations number of steps
        for counter in range(maxIterations):
            # update the gradient
            gradient = self.computeGradient(f, x, h)
            #print('The norm of the gradient is', np.linalg.norm(gradient))
            # stop if the norm of the gradient is near 0
            if np.linalg.norm(gradient) < tolerance:
                print('Gradient descent took', counter, 'iterations to converge')
                print('The norm of the gradient is', np.linalg.norm(gradient))
                # return the approximate critical value x
                return x
            # if we do not converge, print a message
            elif counter == maxIterations-1:
                print("Gradient descent failed")
                print('The gradient is', gradient)
                # return x, sometimes it is still pretty good
                return x

            # take a step in the opposite direction as the gradient with momentum!!
            vdL = B*vdL + (1-B)*gradient
            x -= h*vdL
              
    # compute the gradient
    def computeGradient(self, f, x, h):
        n = len(x)
        gradient = np.zeros(n)

        for counter in range(n):
            xUp = x.copy()
            xUp[counter] += h
            gradient[counter] = (f(xUp) - f(x))/h

        return gradient

# Testing on Mount Pleasant Dataset
For this we use the [Mount Pleasant Data](http://www.hawkeslearning.com/Statistics/dis/datasets.html) which consists of features of homes and their sell values. First we run the without momentum and then we run with momentum. We can see that the number of iterations it takes to converage decreases when we add momentum. 

In [ ]:
from google.colab import drive
import time

# using google colab this is how I imported the data
# first mount the drive
#drive.mount('/content/drive/')

# then import the data from the csv file to an numpy array
path = "/content/sample_data/Mount_Pleasant_Real_Estate_Data.csv"
data = pandas.read_csv(path, sep=',').to_numpy()

# if you are using jupyter notebooks you can use this to import the data
"""
data = pandas.read_csv('data/Mount_Pleasant_Real_Estate_data.csv', sep=',').to_numpy()
print(f'The real estate data\n{data}\nDimensions: {data.shape}')
"""

X = np.array(data[:,2:], dtype=float)
y = np.array(data[:,1], dtype=float)
#print(f'y values\n{y}')
#print(f'X values\n{X}')

# split the data into training and test sets
(trainX, testX, trainY, testY) = train_test_split(X, y, test_size = 0.20, random_state = 1)

# Note that data is scaled and normalized.
trainX = normalize(trainX)
testX = normalize(testX)

trainX = scale(trainX)
testX = scale(testX)

print('****FOR THE GRADIENT-BASED ORDINARY LEAST SQUARES \n')
start = time.time()
# instantiate an OLS model
model = OrdinaryLeastSquaresGradient()

# fit the model to the training data (find the beta parameters)
model.fit(trainX, trainY, h = 0.001, tolerance = 4, maxIterations = 100000)

# return the predicted outputs for the datapoints in the training set
trainPredictions = model.predict(trainX)

# return the predicted outputs for the datapoints in the test set
predictions = model.predict(testX)

end = time.time()
total_time = end-start

# print the coefficient of determination r^2
print('\nThe r^2 score is', r2_score(trainY, trainPredictions))

# print quality metrics
print('The mean absolute error on the training set is', mean_absolute_error(trainY, trainPredictions))

# print quality metrics
print('The mean absolute error on the test set is', mean_absolute_error(testY, predictions))

print('Total time elapsed is', total_time, 'seconds\n')

print('****FOR THE GRADIENT-BASED ORDINARY LEAST SQUARES CODE WITH MOMENTUM \n')
start = time.time()
# instantiate an OLS model
model = OrdinaryLeastSquaresGradientMomentum()

# fit the model to the training data (find the beta parameters)
model.fit(trainX, trainY, h = 0.001, tolerance = 4, maxIterations = 100000)

# return the predicted outputs for the datapoints in the training set
trainPredictions = model.predict(trainX)
# return the predicted outputs for the datapoints in the test set
predictions = model.predict(testX)

end = time.time()
total_time = end-start
# print the coefficient of determination r^2
print('\nThe r^2 score is', r2_score(trainY, trainPredictions))

# print quality metrics
print('The mean absolute error on the training set is', mean_absolute_error(trainY, trainPredictions))

# print quality metrics
print('The mean absolute error on the test set is', mean_absolute_error(testY, predictions))
print('Total time elapsed is', total_time, 'seconds\n')


****FOR THE GRADIENT-BASED ORDINARY LEAST SQUARES 

Gradient descent took 2504 iterations to converge
The norm of the gradient is 3.7822102990306807

The r^2 score is 0.8681363993799857
The mean absolute error on the training set is 79729.73464217168
The mean absolute error on the test set is 68401.06903389767
Total time elapsed is 1.1345787048339844 seconds

****FOR THE GRADIENT-BASED ORDINARY LEAST SQUARES CODE WITH MOMENTUM 

Gradient descent took 2409 iterations to converge
The norm of the gradient is 3.813598474563796

The r^2 score is 0.8681363993800073
The mean absolute error on the training set is 79729.73509409843
The mean absolute error on the test set is 68401.07156949563
Total time elapsed is 1.2796263694763184 seconds



# With Net Elastic Loss
Class that implements an elastic net loss function using gradient descent with momentum for linear regression. This method introduces two hyperparameters: L1 and L2. These values combine the penalties of ridge regression and lasso to get a better loss function. 

In [ ]:
class ElasticLeastSquaresGradient:
        
    # fit the model to the data
    def fit(self, X, y, h, tolerance, maxIterations):
        self.n = X.shape[0]
        self.d = X.shape[1]
        
        # save the training data
        self.data = np.hstack((np.ones([self.n, 1]), X))
        
        # save the training labels
        self.outputs = y
       
        # find the beta values that minimize the sum of squared errors via gradient descent
        X = self.data

        # lambda 1 and 2 for ENR 
        L1= 0.1
        L2= 0.1
      
        # lambda function implements elastic net regression
        L = lambda beta: ((X @ beta).T - y.T) @ (X @ beta - y)+ L1*np.sum(np.abs(np.array(beta)))+L2*(np.array(beta).T@np.array(beta))       
        self.beta = self.gradientDescentMomentum(L, (self.d + 1) * [0], h, tolerance, maxIterations)
                
    # predict the output from testing data
    def predict(self, X):
        # initialize an empty matrix to store the predicted outputs
        yPredicted = np.empty([X.shape[0],1])
        
        # append a column of ones at the beginning of X
        X = np.hstack((np.ones([X.shape[0],1]), X))
        
        # apply the function f with the values of beta from the fit function to each testing datapoint (rows of X)
        for row in range(X.shape[0]):
            yPredicted[row] = self.beta @ X[row,]
            
        return yPredicted

    # run gradient descent to minimize the loss function
    def gradientDescentMomentum(self, f, x0, h, tolerance, maxIterations):
        # set x equal to the initial guess
        x = x0
        vdL = np.zeros(len(x))

        B=0.9
      
        # take up to maxIterations number of steps
        for counter in range(maxIterations):
            # update the gradient
            gradient = self.computeGradient(f, x, h)
           
            # stop if the norm of the gradient is near 0
            if np.linalg.norm(gradient) < tolerance:
                print('Gradient descent took', counter, 'iterations to converge')
                print('The norm of the gradient is', np.linalg.norm(gradient))
                # return the approximate critical value x
                return x
            # if we do not converge, print a message
            elif counter == maxIterations-1:
                print("Gradient descent failed")
                print('The gradient is', gradient)
                # return x, sometimes it is still pretty good
                return x

            # take a step in the opposite direction as the gradient
            
            vdL = B*vdL + (1-B)*gradient
            x -= h*vdL
          
    
    # compute the gradient
    def computeGradient(self, f, x, h):
        n = len(x)
        gradient = np.zeros(n)

        for counter in range(n):
            xUp = x.copy()
            xUp[counter] += h
            gradient[counter] = (f(xUp) - f(x))/h

        return gradient

# Testing on Weather in Szeged Dataset
Using the [Weather in Szeged dataset](https://www.kaggle.com/budincsevity/szeged-weather) and the net elastic loss function from problem 3, predict the temperature. The result using L1 = 0.1 and L2 = 0.1 is a high R^2 value of ~0.9902.

In [ ]:
from google.colab import drive

# first mount the drive
drive.mount('/content/drive/')

# then import the data from the csv file to an numpy array
path = "/content/sample_data/weatherHistory.csv"
data = pandas.read_csv(path, sep=',').to_numpy()

# if you are using jupyter notebooks you can use this to import the data
"""
data = pandas.read_csv('data/weatherHistory.csv', sep=',').to_numpy()
print(f'The real estate data\n{data}\nDimensions: {data.shape}')
"""
X = np.array(data[1:,2:11], dtype=float)
y = np.array(data[1:,1], dtype=float)

#print(f'y values\n{y}')
#print(f'X values\n{X}')

# split the data into training, test, and validation sets
(trainX, testX, trainY, testY) = train_test_split(X, y, test_size = 0.25, random_state = 2)
(trainX, valX, trainY, valY) = train_test_split(trainX, trainY, test_size=0.25, random_state =1) #0.8*0.25=0.2

# Note that data is scaled
trainX = scale(trainX)
testX = scale(testX)
valX = scale(valX)

print('****FOR THE GRADIENT-BASED WITH ELASTIC NET LOSS  \n')
start = time.time()
# instantiate an OLS model
model = ElasticLeastSquaresGradient()

# fit the model to the training data (find the beta parameters)
# should use a very small h value 
model.fit(trainX, trainY, h = 0.000001, tolerance = 0.0001, maxIterations = 100000)

# return the predicted outputs for the datapoints in the training set
trainPredictions = model.predict(trainX)

# return the predicted outputs for the datapoints in the test set
predictions = model.predict(testX)

end = time.time()
total_time = end-start

# print the coefficient of determination r^2
print('\nThe r^2 score is', r2_score(trainY, trainPredictions))

# print quality metrics
print('The mean absolute error on the training set is', mean_absolute_error(trainY, trainPredictions))

# print quality metrics
print('The mean absolute error on the test set is', mean_absolute_error(testY, predictions))
print('Total time elapsed is', total_time, 'seconds\n')

# print quality metrics
predictions = model.predict(valX)
print('The mean absolute error on the validation set is', mean_absolute_error(valY, predictions))





Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
****FOR THE GRADIENT-BASED WITH ELASTIC NET LOSS  

Gradient descent took 428 iterations to converge
The norm of the gradient is 9.680032013893703e-05

The r^2 score is 0.9902035781792737
The mean absolute error on the training set is 0.722750053115553
The mean absolute error on the test set is 0.726523739651633
Total time elapsed is 5.874943256378174 seconds

The mean absolute error on the validation set is 0.7138900372533635
